In [1]:
import pandas as pd

In [14]:
import networkx as nx

### COVID 19 INDIA CROWD-SOURCED DATBASE 
Source - covid19india.org
<br>
Their data sources - https://telegra.ph/Covid-19-Sources-03-19

In [58]:
data = pd.read_csv("COVID19_Raw_Data.csv")

In [59]:
data.head(26)

,Patient Number,State Patient Number,Date Announced,Estimated Onset Date,Age Bracket,Gender,Detected City,Detected District,Detected State,State code,Current Status,Notes,Contracted from which Patient (Suspected),Nationality,Type of transmission,Status Change Date,Source_1,Source_2,Source_3,Backup Notes
0,1,KL-TS-P1,30/01/2020,NaN,20,F,Thrissur,Thrissur,Kerala,KL,Recovered,Travelled from Wuhan,NaN,India,Imported,14/02/2020,https://twitter.com/vijayanpinarayi/status/122...,https://weather.com/en-IN/india/news/news/2020...,NaN,Student from Wuhan
1,2,KL-AL-P1,02/02/2020,NaN,NaN,NaN,Alappuzha,Alappuzha,Kerala,KL,Recovered,Travelled from Wuhan,NaN,India,Imported,14/02/2020,https://www.indiatoday.in/india/story/kerala-r...,https://weather.com/en-IN/india/news/news/2020...,NaN,Student from Wuhan
2,3,KL-KS-P1,03/02/2020,NaN,NaN,NaN,Kasaragod,Kasaragod,Kerala,KL,Recovered,Travelled from Wuhan,NaN,India,Imported,14/02/2020,https://www.indiatoday.in/india/story/kerala-n...,https://twitter.com/ANI/status/122422148580539...,https://weather.com/en-IN/india/news/news/2020...,Student from Wuhan
3,4,DL-P1,02/03/2020,NaN,45,M,East Delhi (Mayur Vihar),East Delhi,Delhi,DL,Recovered,"Travelled from Austria, Italy",NaN,India,Imported,15/03/2020,https://www.indiatoday.in/india/story/not-a-ja...,https://economictimes.indiatimes.com/news/poli...,NaN,Travel history to Italy and Austria
4,5,TS-P1,02/03/2020,NaN,24,M,Hyderabad,Hyderabad,Telangana,TG,Recovered,"Travelled from Dubai to Bangalore on 20th Feb,...",NaN,India,Imported,02/03/2020,https://www.deccanherald.com/national/south/qu...,https://www.indiatoday.in/india/story/coronavi...,https://www.thehindu.com/news/national/coronav...,"Travel history to Dubai, Singapore contact"
5,6,NaN,03/03/2020,NaN,69,M,Jaipur,Italians*,Rajasthan,RJ,Recovered,Travelled from Italy,NaN,Italy,Imported,03/03/2020,https://indianexpress.com/article/india/corona...,https://economictimes.indiatimes.com/news/poli...,https://twitter.com/PTI_News/status/1240878975...,Italian tourist
6,7,NaN,04/03/2020,NaN,55,NaN,Gurugram,Italians*,Haryana,HR,Recovered,Travelled from Italy,P6,Italy,Imported,29/03/2020,https://indianexpress.com/article/india/corona...,https://www.indiatoday.in/india/story/coronavi...,NaN,Italian tourist
7,8,NaN,04/03/2020,NaN,55,NaN,Gurugram,Italians*,Haryana,HR,Recovered,Travelled from Italy,P6,Italy,Imported,29/03/2020,https://indianexpress.com/article/india/corona...,https://www.indiatoday.in/india/story/coronavi...,NaN,Italian tourist
8,9,NaN,04/03/2020,NaN,55,NaN,Gurugram,Italians*,Haryana,HR,Recovered,Travelled from Italy,P6,Italy,Imported,29/03/2020,https://indianexpress.com/article/india/corona...,https://www.indiatoday.in/india/story/coronavi...,NaN,Italian tourist
9,10,NaN,04/03/2020,NaN,55,NaN,Gurugram,Italians*,Haryana,HR,Recovered,Travelled from Italy,P6,Italy,Imported,29/03/2020,https://indianexpress.com/article/india/corona...,https://www.indiatoday.in/india/story/coronavi...,NaN,Italian tourist


In [60]:
data.shape

(8367, 20)

In [61]:
data = data.fillna("-1")

In [62]:
columns = [data.columns]

In [63]:
columns

[Index(['Patient Number', 'State Patient Number', 'Date Announced',
        'Estimated Onset Date', 'Age Bracket', 'Gender', 'Detected City',
        'Detected District', 'Detected State', 'State code', 'Current Status',
        'Notes', 'Contracted from which Patient (Suspected)', 'Nationality',
        'Type of transmission', 'Status Change Date', 'Source_1', 'Source_2',
        'Source_3', 'Backup Notes'],
       dtype='object')]

In [280]:
import copy

In [281]:
data_filtered = data.loc[(data['Contracted from which Patient (Suspected)']!="-1") & ~(data['Contracted from which Patient (Suspected)'].str.startswith("KA"))& ~(data['Contracted from which Patient (Suspected)'].str.startswith(" "))]

In [282]:
data_filtered.shape

(1315, 20)

In [283]:
import re
def extractor(x):
    if x[0]=='E':
        return [1670]
    elif x[0]=='P':
        numbers = re.findall('\d+',x)
        numbers = [int(i) for i in numbers]
        if numbers!=None:
            return numbers
        else:
            return []
    else:
        return []
    

In [284]:
data_filtered['source_cluster_flag'] = data_filtered['Contracted from which Patient (Suspected)'].apply(lambda x:extractor(x))

/home/ashish-j/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [285]:
data_filtered['source_cluster_flag']

6                [6]
7                [6]
8                [6]
9                [6]
10               [6]
11               [6]
12               [6]
13               [6]
14               [6]
15               [6]
16               [6]
17               [6]
18               [6]
19               [6]
20               [6]
21               [4]
22               [4]
23               [4]
24               [4]
25               [4]
26               [4]
27               [6]
35              [35]
36              [35]
37              [35]
38              [35]
39              [29]
46               [4]
47              [22]
48      [35, 36, 37]
            ...     
6624          [1670]
6625          [1670]
6626          [1670]
6627          [1670]
6628          [1670]
6629          [1670]
6630          [1670]
6631          [1670]
6726          [4906]
6727          [4906]
6728          [4906]
6910          [1670]
6911          [1670]
7598          [4905]
7677          [6268]
7678          [1215]
7679         

In [286]:
nodes = list(data_filtered['source_cluster_flag'].index)

In [287]:
nodes

[6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 35,
 36,
 37,
 38,
 39,
 46,
 47,
 48,
 49,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 65,
 66,
 71,
 81,
 83,
 87,
 88,
 90,
 92,
 93,
 97,
 115,
 121,
 122,
 136,
 140,
 141,
 144,
 147,
 154,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 203,
 204,
 206,
 212,
 213,
 214,
 218,
 223,
 224,
 225,
 227,
 228,
 229,
 230,
 231,
 240,
 241,
 242,
 243,
 248,
 249,
 251,
 252,
 260,
 261,
 262,
 263,
 266,
 267,
 278,
 279,
 286,
 289,
 295,
 298,
 303,
 304,
 305,
 306,
 307,
 308,
 310,
 334,
 342,
 343,
 344,
 345,
 353,
 357,
 358,
 359,
 360,
 381,
 387,
 389,
 390,
 391,
 392,
 393,
 396,
 422,
 423,
 426,
 427,
 430,
 431,
 432,
 435,
 436,
 440,
 466,
 472,
 487,
 501,
 514,
 518,
 519,
 527,
 530,
 531,
 532,
 533,
 534,
 553,
 554,
 563,
 566,
 569,
 570,
 573,
 574,
 585,
 596,
 597,
 598,
 599,
 603,
 604,
 605,
 606,
 622,
 630,
 632,
 634,
 

In [288]:
nodes2idx = [(i,j) for i,j in enumerate(nodes)]

In [289]:
def nodes2idx(node):
    return nodes.index(node)

In [290]:
import numpy as np

In [291]:
adjacency_list = {}

In [292]:
for i in nodes:
    adjacency_list[i]=data_filtered['source_cluster_flag'][i]
        

In [293]:
adjacency_list

{6: [6],
 7: [6],
 8: [6],
 9: [6],
 10: [6],
 11: [6],
 12: [6],
 13: [6],
 14: [6],
 15: [6],
 16: [6],
 17: [6],
 18: [6],
 19: [6],
 20: [6],
 21: [4],
 22: [4],
 23: [4],
 24: [4],
 25: [4],
 26: [4],
 27: [6],
 35: [35],
 36: [35],
 37: [35],
 38: [35],
 39: [29],
 46: [4],
 47: [22],
 48: [35, 36, 37],
 49: [35, 36, 37],
 51: [51],
 52: [51],
 53: [43],
 54: [43],
 55: [36, 37, 38],
 56: [36, 37, 38],
 57: [36, 37, 38],
 58: [36, 37, 38],
 59: [44, 45],
 60: [44, 45],
 61: [44, 45],
 65: [44, 45],
 66: [44, 45],
 71: [64],
 81: [30],
 83: [41],
 87: [78],
 88: [78],
 90: [73],
 92: [77],
 93: [77],
 97: [78],
 115: [74],
 121: [95],
 122: [95],
 136: [74],
 140: [1670],
 141: [64],
 144: [146],
 147: [81],
 154: [138],
 160: [160],
 161: [160],
 162: [1670],
 163: [1670],
 164: [1670],
 165: [1670],
 166: [1670],
 167: [1670],
 168: [1670],
 169: [153],
 203: [171],
 204: [171],
 206: [157],
 212: [142],
 213: [142],
 214: [142],
 218: [218],
 223: [172],
 224: [172],
 225: [172

In [294]:
g = nx.from_dict_of_lists(adjacency_list)

In [295]:
g

In [296]:
import matplotlib.pyplot as plt
from pyvis.network import Network

#### central cluster - travelled to Delhi

In [309]:
g1 = Network(height='1000',width='1000',notebook=True)
options = '''{
  "physics": {
    "forceAtlas2Based": {
      "gravitationalConstant": -547,
      "springLength": 275,
      "avoidOverlap": 0.98
    },
    "maxVelocity": 82,
    "minVelocity": 0.87,
    "solver": "forceAtlas2Based"
  }
}'''
g1.set_options(options)
g1.from_nx(g)
g1.show("covid_19.html")